<a href="https://colab.research.google.com/github/BrianGomora/neuralnetwork/blob/main/Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [ ]:
# Libraries for data visualization and analysis
import pandas as pd
import numpy as n
import matplotlib.pyplot as plt
import os
import random 


# Libraries for deep learning 
import keras
from keras.models import Model, Sequential
from keras.layers.core import Flatten,Dense,Dropout,Activation 
from keras.layers import BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D,ZeroPadding2D
#from keras.optimizers import SGD, Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array


import cv2

Mounting drive

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
folder_path = 'gdrive/My Drive/Emotion/'

Mounted at /content/gdrive


IMAGES FROM DISGUST TRAINING DATASET

In [ ]:
image_size = 48

expression = "disgust"
plt.style.use("dark_background")
plt.figure(figsize= (12,12))
for i in range(1,10,1):
  plt.subplot(3,3,i)
  img = load_img(folder_path +"trainDATA/" + expression+ "/" + os.listdir(folder_path + "trainDATA/" + expression)[i], target_size=(image_size, image_size))
  plt.imshow(img)
plt.show()

DATA AUGMENTATION - GENERATING MORE DATA FOR TRAINING for generalization

In [ ]:
batch_size = 16   #number of images per iteration or 64
image_size = 48


imagedatagen_train = ImageDataGenerator(rescale= 1./255,
                                        rotation_range=30,
                                        shear_range=0.3,
                                        zoom_range=0.3,
                                        horizontal_flip= True,
                                        height_shift_range=0.4,
                                        width_shift_range =0.4)

imagedatagen_val = ImageDataGenerator(horizontal_flip = True)

train_imageSET = imagedatagen_train.flow_from_directory(folder_path + "EtrainDATA",
                                                  target_size = (image_size,image_size),
                                                  color_mode = 'grayscale',
                                                  batch_size = batch_size,
                                                  class_mode = "categorical",
                                                  shuffle = True)

test_imageSET = imagedatagen_val.flow_from_directory(folder_path + "EtestDATA",
                                                  target_size = (image_size,image_size),
                                                  color_mode ='grayscale',
                                                  batch_size = batch_size,
                                                  class_mode = "categorical",
                                                  shuffle = False)

BUILDING THE CONVOLUTIONAL NEURAL NETWORK

In [ ]:
num_classes = 7 
model = Sequential()
random.seed()

#1st CNN layer 
model.add(Convolution2D(254,(3,3),padding ="same", input_shape= (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer 
model.add(Convolution2D(252,(3,3),padding ="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Dropout(0.25))

#3Rd CNN layer 
model.add(Convolution2D(512,(3,3),padding ="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Dropout(0.25))

#4th CNN layer 
model.add(Convolution2D(512,(3,3),padding ="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

# Fully connected 1st  Layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected 2nd  Layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(num_classes, activation= 'softmax'))

opt = Adam(lr = 0.2)
model.compile(optimizer = opt,loss='categorical_crossentropy', metrics = ["accuracy"])
model.summary()


TRAINING AND VALIDATION WITH CNN MODEL

In [ ]:
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

epochs = 1000
steps_per_epoch = train_imageSET.n/ train_imageSET.batch_size
validation_steps = test_imageSET.n/ test_imageSET.batch_size



checkpoint = ModelCheckpoint("./model_weights.h5",monitor = "val_loss",
                             save_weights_only = True, mode= 'max',verbose = 1)



reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor = 0.1,patience = 2,min_lr=0.00001,
                              mode='auto')

callback_list = [checkpoint,reduce_lr]

history = model.fit(
    x = train_imageSET,
    steps_per_epoch = steps_per_epoch,
    epochs = epochs, 
    validation_data = test_imageSET,
    validation_steps = validation_steps,
    callbacks = callback_list 
)


PLOT ACCURACY AND LOSS

In [ ]:
plt.style.use('dark_background')

plt.figure(figsize=(20,10))
plt.subplot(1,2,2)
plt.subtitle('Optimizer:Adam', fontsize=10)
plt.ylabel('Loss',fontsize=16)
plt.plot(history.history['loss'],label = 'Training Loss')
plt.plot(history.history['val_loss'],label = 'Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1,2,2)
plt.ylabel('Accuracy'fontsize = 16)
plt.plot(history.history['accuracy'],label = 'Training Accuracy')
plt.plot(history.history['val_accuracy'],label = 'Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

REPRESENT MODEL AS JSON STRING

In [ ]:
fer_json = model.to_json()
with open('fer.json','w') as json_file:
  json_file.write(fer_json)
model.save_weights('fer.h5')